In [ ]:
import yfinance as yf



import datetime


def fetch_latest_data():
   
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    new_data = yf.download('GC=F', interval='1h', start='2023-01-01', end=today)
    return new_data[['Open', 'High', 'Low', 'Close']]  

gold_data_hourly = fetch_latest_data()

print(gold_data_hourly.head())

[*********************100%***********************]  1 of 1 completed

                                  Open         High          Low        Close
Datetime                                                                     
2023-01-03 00:00:00-05:00  1846.900024  1847.599976  1844.199951  1846.800049
2023-01-03 01:00:00-05:00  1846.800049  1850.699951  1846.199951  1848.199951
2023-01-03 02:00:00-05:00  1848.199951  1856.599976  1847.099976  1854.000000
2023-01-03 03:00:00-05:00  1854.099976  1854.400024  1840.500000  1842.099976
2023-01-03 04:00:00-05:00  1842.199951  1847.900024  1840.800049  1845.000000


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


data = gold_data_hourly[['Close']]


scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


time_step = 60  
n_future = 6   

X = []
y = []

for i in range(time_step, len(scaled_data) - n_future + 1):
    X.append(scaled_data[i - time_step:i, 0])  
    y.append(scaled_data[i:i + n_future, 0])    

X = np.array(X)
y = np.array(y)

X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt


seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=n_future)) 


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


history = model.fit(X, y, epochs=10, batch_size=32)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - loss: 0.0214 - mae: 0.0832
Epoch 2/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - loss: 0.0023 - mae: 0.0313
Epoch 3/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 23s 62ms/step - loss: 0.0017 - mae: 0.0261
Epoch 4/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - loss: 0.0014 - mae: 0.0243
Epoch 5/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - loss: 0.0011 - mae: 0.0214
Epoch 6/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - loss: 9.9754e-04 - mae: 0.0202
Epoch 7/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - loss: 8.5585e-04 - mae: 0.0185
Epoch 8/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - loss: 7.9251e-04 - mae: 0.0184
Epoch 9/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 22s 59ms/step - loss: 7.3965e-04 - mae: 0.0177
Epoch 10/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - loss: 6.6525e-04 - mae: 0.0173


In [9]:
# Predict the next 6 hours' price
def predict_next_6_hours(model, last_sequence, scaler):
    last_sequence_scaled = scaler.transform(last_sequence)
    last_sequence_scaled = np.reshape(last_sequence_scaled, (1, last_sequence_scaled.shape[0], 1))
    next_6_hours_prediction_scaled = model.predict(last_sequence_scaled)
    next_6_hours_prediction = scaler.inverse_transform(next_6_hours_prediction_scaled)
    return next_6_hours_prediction[0]

# Get the last 60 hours of data for the next 6 hours' prediction
last_sequence = data[-time_step:].values
next_6_hours_prices = predict_next_6_hours(model, last_sequence, scaler)

print(f"Predicted Gold Prices for the Next 6 Hours: {next_6_hours_prices}")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
Predicted Gold Prices for the Next 6 Hours: [2644.0117 2646.1208 2650.1348 2648.171  2643.416  2648.071 ]


In [12]:
!pip install plotly

In [21]:

import plotly.graph_objects as go
import pandas as pd
fig = go.Figure()

# Historical gold prices trace
x_dates_str = gold_data_hourly.index.strftime('%Y-%m-%d %H:%M:%S').tolist()
y_sequence = list(range(len(x_dates_str)))
z_prices = gold_data_hourly['Close'].values

fig.add_trace(go.Scatter3d(
    x=x_dates_str,
    y=y_sequence,
    z=z_prices,
    mode='lines',
    name='Historical Gold Prices',
    line=dict(color='blue', width=2)
))

# Add predicted next 6 hours trace
predicted_x = pd.date_range(start=gold_data_hourly.index[-1] + pd.Timedelta(hours=1), periods=n_future, freq='H').strftime('%Y-%m-%d %H:%M:%S').tolist()
predicted_y = list(range(y_sequence[-1] + 1, y_sequence[-1] + n_future + 1))

fig.add_trace(go.Scatter3d(
    x=predicted_x,
    y=predicted_y,
    z=next_6_hours_prices,
    mode='lines+markers',
    name='Predicted Next 6 Hours Prices',
    line=dict(color='red', width=2),
    marker=dict(color='red', size=5)
))

# Update layout
fig.update_layout(
    title='Interactive 3D Gold Price Prediction for Next 6 Hours',
    scene=dict(
        xaxis_title='Date',
        yaxis_title='Hour Sequence',
        zaxis_title='Gold Price (USD)',
    ),
    legend=dict(x=0, y=1)
)

# Show plot
fig.show()

In [24]:
import plotly.graph_objects as go
import pandas as pd

In [25]:

# Create 3D plot with Plotly
fig = go.Figure()

# Historical gold prices trace
x_dates_str = gold_data_hourly.index.strftime('%Y-%m-%d %H:%M:%S').tolist()
y_sequence = list(range(len(x_dates_str)))
z_prices = gold_data_hourly['Close'].values

hover_texts_historical = [f"Date: {x}, Hour: {y}" for x, y in zip(x_dates_str, y_sequence)]

fig.add_trace(go.Scatter3d(
    x=x_dates_str,
    y=y_sequence,
    z=z_prices,
    mode='lines',
    name='Historical Gold Prices',
    line=dict(color='blue', width=2),
    hovertemplate='%{text}<br>Price: %{z:.2f}',
    text=hover_texts_historical
))

# Add predicted next 6 hours trace
predicted_x = pd.date_range(start=gold_data_hourly.index[-1] + pd.Timedelta(hours=1), periods=n_future, freq='H').strftime('%Y-%m-%d %H:%M:%S').tolist()
predicted_y = list(range(y_sequence[-1] + 1, y_sequence[-1] + n_future + 1))

hover_texts_predicted = [f"Date: {x}, Hour: {y}" for x, y in zip(predicted_x, predicted_y)]

fig.add_trace(go.Scatter3d(
    x=predicted_x,
    y=predicted_y,
    z=next_6_hours_prices,
    mode='lines+markers',
    name='Predicted Next 6 Hours Prices',
    line=dict(color='red', width=2),
    marker=dict(color='red', size=5),
    hovertemplate='%{text}<br>Price: %{z:.2f}',
    text=hover_texts_predicted
))

# Update layout
fig.update_layout(
    title='Interactive 3D Gold Price Prediction for Historical and Next 6 Hours',
    scene=dict(
        xaxis_title='Date and Time',
        yaxis_title='Hour Sequence (Historical & Predicted)',
        zaxis_title='Gold Price (USD)',
    ),
    legend=dict(x=0, y=1)
)

# Show plot
fig.show()

In [20]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Prepare data for plotting
x_dates = gold_data_hourly.index[-100:]  # Last 100 hours of date
y_sequence = np.arange(100)  # Sequence from 0 to 99
z_prices = data['Close'].tail(100).values  # Historical gold prices

# Convert dates to string format for plotting in plotly
x_dates_str = x_dates.strftime('%Y-%m-%d %H:%M:%S')

# Create 3D plot
fig = go.Figure()

# Add historical gold prices trace
fig.add_trace(go.Scatter3d(
    x=x_dates_str,  # Dates for X-axis
    y=y_sequence,   # Hour sequence for Y-axis
    z=z_prices,     # Gold prices for Z-axis
    mode='lines',
    name='Historical Gold Prices',
    line=dict(color='blue', width=2)
))

# Add predicted next 6 hours price trace
predicted_x = pd.date_range(start=gold_data_hourly.index[-1] + pd.Timedelta(hours=1), periods=6, freq='H').strftime('%Y-%m-%d %H:%M:%S').tolist()  # Next 6 hours' dates
predicted_y = np.arange(y_sequence[-1] + 1, y_sequence[-1] + 7)  # Next 6 hour sequences
predicted_z = next_6_hours_prices  # Predicted prices for next 6 hours

fig.add_trace(go.Scatter3d(
    x=predicted_x,  # Dates for the next 6 hours
    y=predicted_y,  # Sequence for the next 6 hours
    z=predicted_z,  # Predicted gold prices for the next 6 hours
    mode='markers+lines',
    name='Predicted Next 6 Hours Prices',
    line=dict(color='red', width=2),
    marker=dict(color='red', size=8)
))

# Update layout
fig.update_layout(
    title='Interactive 3D Gold Price Prediction for the Next 6 Hours',
    scene=dict(
        xaxis_title='Date',
        yaxis_title='Hour Sequence',
        zaxis_title='Gold Price (USD)',
    ),
    legend=dict(x=0, y=1)
)

# Show plot
fig.show()
